In [16]:
import os
import tensorflow as tf
import numpy as np
import pathlib
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import joblib

# Dataset path
data_dir = pathlib.Path('/content/drive/MyDrive/Mushrooms')
class_names = sorted([item.name for item in data_dir.iterdir() if item.is_dir()])
class_to_idx = {name: idx for idx, name in enumerate(class_names)}

# Check and load valid images
def is_valid_image(filepath):
    try:
        img = tf.io.read_file(filepath)
        tf.image.decode_jpeg(img)
        return True
    except tf.errors.InvalidArgumentError:
        return False

valid_images, valid_labels = [], []
for cls in class_names:
    for path in (data_dir / cls).glob("*.jpg"):
        if is_valid_image(str(path)):
            valid_images.append(str(path))
            valid_labels.append(class_to_idx[cls])

dataset = tf.data.Dataset.from_tensor_slices((valid_images, valid_labels))
dataset = dataset.shuffle(len(valid_images), seed=123)
train_size = int(0.8 * len(valid_images))
train_ds = dataset.take(train_size)
val_ds = dataset.skip(train_size)

def preprocess(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

train_ds = train_ds.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

# ===  Data Augmentation Layer  ===
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.01),
    tf.keras.layers.RandomZoom(0.01),
])

# Step 1: Build + Train CNN with softmax using Functional API
from tensorflow.keras import layers, models

inputs = layers.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)  # tiny augmentation applied only here
x = layers.Conv2D(32, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(len(class_names), activation='softmax')(x)

full_model = models.Model(inputs, outputs)

full_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

full_model.fit(train_ds, epochs=30, validation_data=val_ds)

# Step 2: Create feature extractor (cut off softmax)
feature_extractor = tf.keras.Model(
    inputs=full_model.input,
    outputs=full_model.layers[-2].output  # second-to-last layer (Dense(128))
)

# Step 3: Extract features
def extract_features(dataset, extractor):
    features, labels = [], []
    for imgs, lbls in dataset:
        feats = extractor.predict(imgs, verbose=0)
        features.append(feats)
        labels.append(lbls.numpy())
    return np.vstack(features), np.concatenate(labels)

train_features, train_labels = extract_features(train_ds, feature_extractor)
val_features, val_labels = extract_features(val_ds, feature_extractor)

# Step 4: Train SVM on extracted features
svm_clf = SVC(kernel='linear')
svm_clf.fit(train_features, train_labels)

# Step 5: Evaluate SVM
val_preds = svm_clf.predict(val_features)
accuracy = accuracy_score(val_labels, val_preds)
print(f"SVM classifier accuracy on validation set: {accuracy * 100:.2f}%")



Epoch 1/30
169/169 ━━━━━━━━━━━━━━━━━━━━ 11s 57ms/step - accuracy: 0.2335 - loss: 2.0864 - val_accuracy: 0.3252 - val_loss: 1.8098
Epoch 2/30
169/169 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - accuracy: 0.3613 - loss: 1.7722 - val_accuracy: 0.4209 - val_loss: 1.6607
Epoch 3/30
169/169 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - accuracy: 0.4386 - loss: 1.6061 - val_accuracy: 0.5093 - val_loss: 1.4396
Epoch 4/30
169/169 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step - accuracy: 0.4862 - loss: 1.4954 - val_accuracy: 0.5130 - val_loss: 1.3785
Epoch 5/30
169/169 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - accuracy: 0.5400 - loss: 1.3576 - val_accuracy: 0.5397 - val_loss: 1.2805
Epoch 6/30
169/169 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - accuracy: 0.5520 - loss: 1.3261 - val_accuracy: 0.5820 - val_loss: 1.2440
Epoch 7/30
169/169 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - accuracy: 0.6182 - loss: 1.1449 - val_accuracy: 0.6578 - val_loss: 1.0253
Epoch 8/30
169/169 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - accuracy: 0.6393 - loss: 1.0880 - v

In [13]:
# Evaluate CNN directly on validation set
cnn_eval = full_model.evaluate(val_ds)
print(f"CNN softmax accuracy on validation set: {cnn_eval[1] * 100:.2f}%")


43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 0.9813 - loss: 0.0732
CNN softmax accuracy on validation set: 98.14%


In [14]:
#  Save models
full_model.save('trained_cnn_model.keras')
joblib.dump(svm_clf, 'trained_svm_classifier.joblib')

['trained_svm_classifier.joblib']